In [102]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from rasterio.coords import BoundingBox
from rasterio.transform import Affine
from shapely.geometry import mapping

In [61]:
geotiff_dtm = "data/M_25DN2.TIF"
geotiff_dsm = "data/R_25DN2.TIF"
geotiff_chm = "output/CHM_25DN2.tiff"
buildings = "data/pand.gpkg"

# with rasterio.open(geotiff_path) as src:
#     bounds = src.bounds  # BoundingBox(left, bottom, right, top)
#     crs = src.crs  


In [62]:
# bbox_rd_new = (bounds.left, bounds.bottom, bounds.right, bounds.top)
# wfs_bag = 'https://service.pdok.nl/lv/bag/wfs/v2_0'
# layer = 'bag:pand'

In [63]:
# Connect to the WFS server
# 
# wfs = WebFeatureService(url=wfs_bag, version='2.0.0', parse_remote_metadata=False)
# srs = 'EPSG:28992' 
# 
# capabilities = wfs.getcapabilities()
# 
# response = wfs.getfeature(typename=layer,
#                           bbox=bbox_rd_new + (srs,),  # Include EPSG code for RD New
#                           outputFormat='application/json; subtype=geojson')
# 
# response_text = response.read()
# 
# print(response_text[:500])
# gdf = gpd.read_file(io.BytesIO(response.read()))
# 
# print(gdf.head())

In [64]:
# Check the GeoDataFrame
# print(gdf.head())

# geometry_counts = gdf.geometry.geom_type.value_counts()
# print("Geometry type counts:")
# print(geometry_counts)

In [65]:
def get_overlapping_bbox(raster_paths):
    bboxes = []

    for raster_path in raster_paths:
        with rasterio.open(raster_path) as src:
            bbox = src.bounds 
            bboxes.append(bbox)
    
    left = max([bbox.left for bbox in bboxes])
    bottom = max([bbox.bottom for bbox in bboxes])
    right = min([bbox.right for bbox in bboxes])
    top = min([bbox.top for bbox in bboxes])

    return BoundingBox(left, bottom, right, top)

def crop_raster_to_bbox(raster_path, bbox, output_path, no_data=-9999):
    with rasterio.open(raster_path) as src:
        window = src.window(bbox.left, bbox.bottom, bbox.right, bbox.top)

        cropped_data = src.read(window=window)
        
        if src.nodata is not None:
            cropped_data = np.where(cropped_data == src.nodata, no_data, cropped_data)
        cropped_data = np.where(np.isinf(cropped_data) | np.isnan(cropped_data), no_data, cropped_data)

        transform = src.window_transform(window)
        with rasterio.open(
            output_path,
            'w',
            driver=src.driver,
            height=cropped_data.shape[1],
            width=cropped_data.shape[2],
            count=src.count,
            dtype=cropped_data.dtype,
            crs=src.crs,
            transform=transform,
            nodata=no_data
        ) as dst:
            dst.write(cropped_data)


In [66]:
raster_paths = [geotiff_dtm, geotiff_dsm, geotiff_chm]

# Get the overlapping bounding box
overlapping_bbox = get_overlapping_bbox(raster_paths)

# Crop each raster to the overlapping bounding box
output_dsm = "output/cropped_R_25DN2.TIF"
output_dtm = "output/cropped_M_25DN2.TIF"
output_chm = "output/cropped_CHM_25DN2.TIF"

crop_raster_to_bbox(geotiff_dtm, overlapping_bbox, output_dtm)
crop_raster_to_bbox(geotiff_dsm, overlapping_bbox, output_dsm)
crop_raster_to_bbox(geotiff_chm, overlapping_bbox, output_chm)

In [67]:
def buildings_cutout(raster_path, buildings_path, output_path, nodata_value=-9999):
    buildings = gpd.read_file(buildings_path)
    building_geometries = [mapping(geom) for geom in buildings.geometry]

    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, building_geometries, invert=False, nodata=nodata_value)

        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": nodata_value
        })

        with rasterio.open(output_path, "w", **out_meta) as dst:
            dst.write(out_image)


In [68]:
output_buildings = "output/buildings_25DN2.TIF"
buildings_cutout(output_dsm, buildings, output_buildings)

In [69]:
from rasterio.fill import fillnodata

In [70]:
def fill_dsm(raster_path, output_path, max_search_distance=80.0, smoothing_iterations=2):
    with rasterio.open(raster_path) as src:
        array = src.read(1)  
        nodata = src.nodata
        transform = src.transform
        crs = src.crs
        nrows, ncols = array.shape
    
        filled_array = fillnodata(array, mask=src.read_masks(1), max_search_distance=max_search_distance, smoothing_iterations=smoothing_iterations)

        with rasterio.open(raster_path) as src:
            with rasterio.open(output_path, 'w', driver='GTiff', height=nrows, width=ncols, count=1,
                               dtype=filled_array.dtype, nodata=nodata, crs=crs, transform=transform) as dst:
                dst.write(filled_array, 1)


In [84]:
fill_dsm(output_dtm,"output/filled_M_25DN2.TIF")

In [88]:
def chm_finish(chm, dtm, output_path):
    with rasterio.open(chm) as chm_src, rasterio.open(dtm) as dtm_src:
        dtm_array = dtm_src.read(1)
        chm_array = chm_src.read(1)
        # 
        # if chm_src.nodata is not None:
        #     chm_array[chm_array == chm_src.nodata] = 0

        result_array = chm_array - dtm_array
        
        result_array[result_array < 0] = 0

        # result_array = np.where(np.isnan(result_array) | np.isinf(result_array), 0 , result_array)

        # Save the result raster
        with rasterio.open(output_path, 'w', driver='GTiff', height=result_array.shape[0], width=result_array.shape[1],
                           count=1, dtype=result_array.dtype, nodata=0, crs=dtm_src.crs, transform=dtm_src.transform) as dst:
            dst.write(result_array, 1)

In [95]:
chm_finish(output_chm, "output/filled_M_25DN2.TIF", "output/fin_CHM_25DN2.TIF")

In [91]:
def replace_buildings(filled, buildings, output_path, nodata_value=-9999):
    with rasterio.open(filled) as filled_src, rasterio.open(buildings) as buildings_src:
        filled_array = filled_src.read(1)
        buildings_array = buildings_src.read(1)
        nodata_filled = filled_src.nodata
        nodata_buildings = buildings_src.nodata
 
        filled_array = np.where(filled_array == nodata_filled, nodata_value, filled_array)
        buildings_array = np.where(buildings_array == nodata_buildings, nodata_value, buildings_array)

        building_mask = buildings_array != nodata_value

        result_array = np.where(building_mask, buildings_array, filled_array)

        out_meta = filled_src.meta.copy()
        out_meta.update({
            'dtype': result_array.dtype,
            'nodata': nodata_value
        })

        with rasterio.open(output_path, 'w', **out_meta) as dst:
            dst.write(result_array, 1)

In [92]:
replace_buildings("output/filled_M_25DN2.TIF", "output/buildings_25DN2.TIF", "output/fin_DTM_25DN2.TIF", nodata_value=-9999)

In [111]:
def split_raster(raster_path, output_top_path, output_bottom_path):
     with rasterio.open(raster_path) as src:
        # Read the data
        data = src.read(1)  # Read the first band
        transform = src.transform
        crs = src.crs
        nodata = src.nodata
        
        nrows, ncols = data.shape
        split_row = nrows // 2

        top_half = data[:split_row, :]
        bottom_half = data[split_row:, :]

        top_transform = transform
        bottom_transform = transform * Affine.translation(0, -split_row * 2 *transform.e)

        top_meta = src.meta.copy()
        top_meta.update({
            'height': top_half.shape[0],
            'width': top_half.shape[1],
            'transform': top_transform,
            'nodata': nodata
        })

        bottom_meta = src.meta.copy()
        bottom_meta.update({
            'height': bottom_half.shape[0],
            'width': bottom_half.shape[1],
            'transform': bottom_transform,
            'nodata': nodata
        })

        with rasterio.open(output_top_path, 'w', **top_meta) as top_dst:
            top_dst.write(top_half, 1)

        with rasterio.open(output_bottom_path, 'w', **bottom_meta) as bottom_dst:
            bottom_dst.write(bottom_half, 1)


In [113]:
split_raster("output/fin_DTM_25DN2.TIF", "final/top_DTM_25DN2.TIF", "final/bottom_DTM_25DN2.TIF")
split_raster("output/fin_CHM_25DN2.TIF", "final/top_CHM_25DN2.TIF", "final/bottom_CHM_25DN2.TIF")

Top half saved to final/top_DTM_25DN2.TIF
Bottom half saved to final/bottom_DTM_25DN2.TIF
Top half saved to final/top_DTM_25DN2.TIF
Bottom half saved to final/bottom_DTM_25DN2.TIF
Top half saved to final/top_CHM_25DN2.TIF
Bottom half saved to final/bottom_CHM_25DN2.TIF
Top half saved to final/top_CHM_25DN2.TIF
Bottom half saved to final/bottom_CHM_25DN2.TIF
